In [60]:
# Define the frame size in bytes
frame_size = 1024

# Open the binary file and read its content
with open('/content/drive/MyDrive/Colab Notebooks/OCR/example.pdf', 'rb') as file:
    binary_content = file.read()

print("binary_content : ", binary_content)

# Extract frames from the binary data
frames = [binary_content[i:i+frame_size] for i in range(0, len(binary_content), frame_size)]

print("frames : ", frames)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [61]:
# type(frames)

In [62]:
# frames[0], len(frames)

In [ ]:
import cv2
import numpy as np

# Define the frame size in pixels
frame_width = 640
frame_height = 480

# Create a VideoWriter object
video_writer = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

# Define the output directory
output_dir = 'frames/'

# Loop through the frames and write them to the video file
for i, frame in enumerate(frames):
    # Convert the binary frame to a numpy array
    np_frame = np.frombuffer(frame, np.uint8)
    # print(i, np_frame, len(np_frame))
    if len(np_frame) != frame_size:
        extra_bits = [0]*(frame_size-len(np_frame))
        np_frame = np.append(np_frame, extra_bits)
        # print(len(np_frame))
    try: 
        np_frame = np.reshape(np_frame, (frame_height, frame_width, 3))

        # Define the output file name
        output_file = f'{output_dir}/frame_{i:04}.png'
        
        # Save the frame as an image file
        cv2.imwrite(output_file, np_frame)
        
        # Write the frame to the video file
        video_writer.write(np_frame)
    except Exception as e:
      print(f"Error at {i} - {e}. ")

# Release the VideoWriter object
video_writer.release()


In [5]:
import numpy as np

# Define the frame size in pixels
frame_width = 640
frame_height = 480

# Define the number of color channels
num_channels = 3

# Extract a single frame from the binary data
binary_frame = binary_data[start_index:end_index]

# Convert the binary frame to a numpy array
np_frame = np.frombuffer(binary_frame, dtype=np.uint8)

# Reshape the numpy array to match the frame size and number of channels
np_frame = np_frame.reshape(frame_height, frame_width, num_channels)


NameError: ignored

In [20]:
480*640*3

921600

In [21]:
1024*438

448512

In [81]:
import cv2
import numpy as np
import subprocess
import os

def pdf_to_video(pdf_path, video_path):
    # Define the output directory for the PNG frames
    frames_dir = 'frames'
    os.makedirs(frames_dir, exist_ok=True)

    file_size = os.path.getsize(pdf_path)
    print(f'The size of {pdf_path} is {file_size} bytes.')

    # # Convert the PDF to a binary string using subprocess
    # result = subprocess.run(['pdftotext', '-raw', pdf_path, '-'], capture_output=True)
    # pdf_text = result.stdout.decode('utf-8')

    # # Convert the binary string to binary data
    # binary_data = pdf_text.encode('utf-8')

    # Open the binary file and read its content
    with open(pdf_path, 'rb') as file:
        binary_data = file.read()
    # print(f"type : {type(binary_data)} | length : {len(binary_data)} ")

    # Define the frame size in pixels
    frame_width = 640
    frame_height = 480

    # Define the number of color channels
    num_channels = 3

    # Calculate the total number of frames
    # print(f"{len(binary_data)} // {(frame_width * frame_height * num_channels)}")
    total_frames = len(binary_data) // (frame_width * frame_height * num_channels)
    print(f"total_frames : {total_frames}")

    # Define the output video codec and frame rate
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 30.0

    # Create a VideoWriter object to write the frames to an MP4 file
    video_writer = cv2.VideoWriter(video_path, fourcc, fps, (frame_width, frame_height))

    # Loop through the frames and write them to the video file
    for i in range(total_frames):
        # Extract a single frame from the binary data
        start_index = i * frame_width * frame_height * num_channels
        end_index = start_index + frame_width * frame_height * num_channels
        binary_frame = binary_data[start_index:end_index]

        # Convert the binary frame to a numpy array
        np_frame = np.frombuffer(binary_frame, dtype=np.uint8)

        # Reshape the numpy array to match the frame size and number of channels
        np_frame = np_frame.reshape(frame_height, frame_width, num_channels)
        # print(f"np_frame: {np_frame}")

        # Write the frame to the video file
        video_writer.write(np_frame)

        # Save the frame as a PNG image file
        frame_file = f'{frames_dir}/frame_{i:04}.png'
        cv2.imwrite(frame_file, np_frame)
        # print(f"Done with : {frame_file}")

    # Release the VideoWriter object and close the file
    video_writer.release()

pdf_path = "/content/resume.csv"
# "/content/santhosh resume.pdf"
# "/content/drive/MyDrive/Colab Notebooks/OCR/example.pdf"
video_path = "output1.mp4"
pdf_to_video(pdf_path, video_path)

The size of /content/resume.csv is 5913933 bytes.
total_frames : 6


In [82]:
!zip -r frames.zip frames/

updating: frames/ (stored 0%)
updating: frames/frame_0005.png (deflated 4%)
updating: frames/frame_0004.png (deflated 4%)
updating: frames/frame_0002.png (deflated 3%)
updating: frames/frame_0001.png (deflated 5%)
updating: frames/frame_0000.png (deflated 4%)
updating: frames/frame_0003.png (deflated 3%)


In [80]:
# !rm -r frames/